In [2]:
from glob import glob
import shutil
import os
import dicom2nifti
import nibabel
import numpy as np
import pydicom as dicom

In [3]:
# deleting empty

In [4]:
to_empty_path = 'D:/AI_Imaging/Cancer_Recognision/Liver1/dicom_files'

In [5]:
def delet_worthless_data(path):
                list_labels_all = glob(path + '/labels/*')
                empty_dcoms = {}

                #listing all labels data 
                for folder in list_labels_all:
                        list_of_fotos = glob(folder + '/*.dcm')
                        empty_dcoms[folder] = []
                        for foto in list_of_fotos:
                                dcm = dicom.dcmread(foto).pixel_array
                                np_unique = np.unique(dcm)
                                if len(np_unique) == 1:
                                        empty_dcoms["{0}".format(folder)].append(foto)

                #normalizing labels name data
                empty_dcoms_norm = {}
                for i in empty_dcoms:
                        helper = i.split('\\')
                        empty_dcoms_norm[helper[1]] = []
                        for dcom in empty_dcoms[i]:
                                split = dcom.split('\\')
                                empty_dcoms_norm[helper[1]].append(split[2])

                #deleting worhless ones
                for folder in empty_dcoms_norm:
                        to_empty_path_img = path + '/img/' + folder +'/'
                        to_empty_path_labels = path + '/labels/' + folder +'/'
                        for name in empty_dcoms_norm[folder]:
                                os.remove(os.path.normpath(to_empty_path_img + name))
                                os.remove(os.path.normpath(to_empty_path_labels + name))

In [6]:
delet_worthless_data(to_empty_path)

In [7]:
### We need nifties with same or close amount of slices per photo otherwise we will lose a lot of data.

In [8]:
# conversion dcom files into groups of 37 dicoms (slices)

In [9]:
number_of_slices = 37
files_labels_input_path = "D:/AI_Imaging/Cancer_Recognision/Liver1/dicom_files/labels"
groups_labels_output_path = "D:/AI_Imaging/Cancer_Recognision/Liver1/dicom_groups/labels"
files_img_input_path = "D:/AI_Imaging/Cancer_Recognision/Liver1/dicom_files/img"
groups_img_output_path = "D:/AI_Imaging/Cancer_Recognision/Liver1/dicom_groups/img"

In [10]:
def split_function(input, output):
                for folder in glob(input + '/*'):
                        folder_name = os.path.basename(os.path.normpath(folder))
                        slice_ratio = int(len(glob(folder + '/*.dcm'))/number_of_slices)
                        #dividing dicoms into new folders - to have equal parts
                        for i in range(slice_ratio):
                                output_path_name = os.path.join(output, folder_name + '_' + str(i))
                                os.mkdir(output_path_name)
                                for i, file in enumerate(glob(folder + '/*.dcm')):
                                                if i == number_of_slices:
                                                        break
                                                shutil.move(file, output_path_name)


In [11]:
split_function(files_img_input_path, groups_img_output_path)
split_function(files_labels_input_path, groups_labels_output_path)

In [12]:
# conversion to nifti files

In [13]:
#!pip install dicom2nifti

In [14]:
groups_img_input_path = groups_img_output_path + '/*'
groups_labels_input_path = groups_labels_output_path + '/*'
nifti_img_out_path = 'D:/AI_Imaging/Cancer_Recognision/Liver1/nifti_final_files/img/'
nifti_labels_out_path = 'D:/AI_Imaging/Cancer_Recognision/Liver1/nifti_final_files/labels/'

list_img = glob(groups_img_input_path)
list_labels = glob(groups_labels_input_path)

In [15]:
def conv_dicom2nifti(list, output):
        for foto in list:
                foto_name = os.path.basename(os.path.normpath(foto))
                dicom2nifti.dicom_series_to_nifti(foto, os.path.join(output, foto_name + '.nii'))

In [16]:
conv_dicom2nifti(list_img, nifti_img_out_path)
conv_dicom2nifti(list_labels, nifti_labels_out_path)

In [17]:
# splitting data 

In [18]:
splitted_nifti_out_path = 'D:/AI_Imaging/Cancer_Recognision/Liver1/splitted_nifti_final_files/'

In [30]:
def split_data(img_path, label_path, train_size):
    img_nifti_list = sorted(glob(img_path + "\*"))
    lab_nifti_list = sorted(glob(label_path + "\*"))
    nifti_len = len(img_nifti_list)
    
    for i, (img, lab) in enumerate(zip(img_nifti_list, lab_nifti_list)):
        if i/nifti_len <= train_size:
            shutil.copy(img, os.path.join(splitted_nifti_out_path, "train_img"))
            shutil.copy(lab, os.path.join(splitted_nifti_out_path, "train_labels"))
        else:
            shutil.copy(img, os.path.join(splitted_nifti_out_path, "test_img"))
            shutil.copy(lab, os.path.join(splitted_nifti_out_path, "test_labels"))


In [33]:
split_data(nifti_img_out_path, nifti_labels_out_path, 0.7)